In [2]:
!pip install spacy==3.0.0
!pip install ml-datasets
!python -m spacy download en_core_web_md

  Using cached spacy-3.0.0.tar.gz (7.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe' 'C:\Users\emely\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' get_requires_for_build_wheel 'C:\Users\emely\AppData\Local\Temp\tmpg3m1ux7i'
       cwd: C:\Users\emely\AppData\Local\Temp\pip-install-_8whz8bq\spacy_01b71bca0fe347c8b24cf74e6966bb47
  Complete output (164 lines):
  Copied C:\Users\emely\AppData\Local\Temp\pip-install-_8whz8bq\spacy_01b71bca0fe347c8b24cf74e6966bb47\setup.cfg -> C:\Users\emely\AppData\Local\Temp\pip-install-_8whz8bq\spacy_01b71bca0fe347c8b24cf74e6966bb47\spacy\tests\package
  Copied C:\Users\emely\AppData\Local\Temp\pip-install-_8whz8bq\spacy_01b71bca0fe347c8b24cf74e6966bb47\pyproject.toml -> C:\Users\emely\AppData\Local\Temp\pip-install-_8whz8bq\spacy_01b71bca0fe347c8b24cf74e6966bb47\spacy\tests\package
  Cythonizing sources
  Compiling spacy/training/example.pyx because it 

You should consider upgrading via the 'C:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


c:\Users\emely\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
84131840it [00:15, 5330816.83it/s]                               


Untaring file...


In [ ]:
import spacy
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from ml_datasets import imdb
train_data, valid_data = imdb()
nlp = spacy.load("en_core_web_md")

In [3]:
train_data[0][1]

'pos'

In [4]:
nlp.pipe_names

# wec2vec - вектор смещения (измерение описывает какой оспект) (по контексту делает вектор слова)
# http://jalammar.github.io/illustrated-word2vec/
# tok2vec - вектор смещения (измерение описывает какой оспект) (по контексту делает вектор слова)
# tagger
# parser
# attribute_ruler
# lemmatizer приводит слово в нормальную форму
# ner

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [5]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) намного быстрее, чем запуск
    # nlp(text) для каждого текста
    # as_tuples позволяет нам передавать кортеж,
    # первый из которых обрабатывается как текст
    # второй будет возвращен таким, какой он есть.
    a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 'neg':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [6]:
num_texts = 100 # берем 100 текстов
train_docs = make_docs(train_data[:num_texts])

  0%|          | 0/100 [00:02<?, ?it/s]


In [15]:
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

  0%|          | 0/100 [00:02<?, ?it/s]


https://spacy.io/usage/training#quickstart

base_config.cfg

# что здесь происходит: мы заполнили base_config, а эта команда на его основе прописывает config.cfg

! python -m spacy init fill-config base_config.cfg config.cfg

! python -m spacy train config.cfg --output ./output

In [16]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [17]:
! python -m spacy train config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       28.06    0.28
  2     200         49.80       51.00    0.51
  4     400          4.41       60.10    0.60
  6     600          0.00       58.80    0.59
  8     800          0.00       52.43    0.52
 10    1000          0.00       56.59    0.57
 12    1200          0.00       54.84    0.55
 14    1400          0.00       56.01    0.56
 16    1600          0.00       54.84    0.55
 18    1800          0.00       56.01    0.56
 20    2000          0.00       54.84    0.55
✔ Saved pipeline to output directory
output\model-last


сгенерили крч модеели

In [18]:
nlp = spacy.load("output/model-best")
text=""

while text != 'quit':
    text = input("Введи текст - ")
    doc = nlp(text)
    if doc.cats['positive'] > .5:
        print('yes')
    else:
        print('no')



no
yes
no
no


![Alt text](image.png)